In [1]:
import fitz  # PyMuPDF

def extract_text_and_links(pdf_path: str):
    full_text = ""
    links = []

    pdf = fitz.open(pdf_path)
    
    for page_num, page in enumerate(pdf, start=1):
        full_text += page.get_text()

        # Extract links on this page
        for link in page.get_links():
            uri = link.get("uri", None)
            rect = link.get("from", None)
            if uri and rect:
                # Get anchor text behind the link
                anchor_text = page.get_textbox(rect)
                links.append({
                    "page": page_num,
                    "text": anchor_text.strip(),
                    "url": uri.strip()
                })

    pdf.close()

    return full_text, links

In [2]:
if __name__ == "__main__":
    path = "Pandharinath_resume.pdf"
    text, links = extract_text_and_links(path)

    print("=== Extracted Text ===")
    print(text[:500])

    print("\n=== Extracted Links ===")
    for link in links:
        print(f"[{link['text']}]({link['url']}) on page {link['page']}")

=== Extracted Text ===
Pandharinath Maske
pandharinath1437@gmail.com
•
+91 8010922929
•
LinkedIn
•
GitHub
•
LeetCode
Education
T.Y B.Tech in Computer Science (AI), VIIT Pune
CGPA: 7.56
Graduation: 2027
Projects
AI-Powered Coffee Shop Chatbot
[GitHub]
[Live Demo]
June 2025 – Present
• Built a modular multi-agent chatbot using LangGraph to handle coffee shop tasks like orders, updates,
and general queries.
• Created custom tools for menu retrieval, order management, and user interaction, integrated into agent
workflows.

=== Extracted Links ===
[pandharinath1437@gmail.com](mailto:pandhari2527@gmail.com) on page 1
[+91 8010922929](tel:+918010922929) on page 1
[LinkedIn](https://www.linkedin.com/in/pandharinath-maske-538381289) on page 1
[GitHub](https://github.com/Pandharimaske) on page 1
[LeetCode](https://leetcode.com/u/PandhariMaske1437/) on page 1
[[GitHub]](https://github.com/Pandharimaske/Coffee_Shop_ChatBot) on page 1
[[Live Demo]](https://coffee-shop-chatbot-p7tr.onrender.com) on 

In [8]:
resume_text_data = text + str(links)

In [10]:
print(resume_text_data)

Pandharinath Maske
pandharinath1437@gmail.com
•
+91 8010922929
•
LinkedIn
•
GitHub
•
LeetCode
Education
T.Y B.Tech in Computer Science (AI), VIIT Pune
CGPA: 7.56
Graduation: 2027
Projects
AI-Powered Coffee Shop Chatbot
[GitHub]
[Live Demo]
June 2025 – Present
• Built a modular multi-agent chatbot using LangGraph to handle coffee shop tasks like orders, updates,
and general queries.
• Created custom tools for menu retrieval, order management, and user interaction, integrated into agent
workflows.
• Implemented long-term (user profile) and short-term (session summary) memory using Supabase and
Pinecone.
• Used Agentic RAG with BGE embeddings for contextual grounding and semantic search.
Tech Stack: Python, FastAPI, LangGraph, LangChain, Pinecone, Supabase, Hugging Face, Groq
(LLaMA3), BGE Embeddings
LLM-Based Book Recommender
[GitHub]
May 2025
• Developed an intelligent book recommendation system leveraging SentenceTransformer embeddings
and ChromaDB for semantic search.
• Designed a que

In [3]:
from pydantic import BaseModel, HttpUrl, EmailStr
from typing import Optional, List


class Project(BaseModel):
    project_name: str
    github_link: Optional[HttpUrl]
    live_link: Optional[HttpUrl]
    time_period: Optional[str]
    features: Optional[List[str]] = []
    tech_stack: Optional[List[str]] = []


class SkillsSection(BaseModel):
    programming_languages: Optional[List[str]] = []
    frameworks: Optional[List[str]] = []
    libraries_tools: Optional[List[str]] = []
    databases: Optional[List[str]] = []
    soft_skills: Optional[List[str]] = []


class ResumeAgentState(BaseModel):
    # Personal Info
    name: Optional[str]
    email: Optional[EmailStr]
    mob_no: Optional[str]

    # Profile Links
    linkedin: Optional[HttpUrl]
    github: Optional[HttpUrl]
    leetcode: Optional[HttpUrl]

    # Skills and Projects
    skills: Optional[SkillsSection] = SkillsSection()
    projects: Optional[List[Project]] = []

In [4]:
from pydantic import BaseModel, EmailStr, HttpUrl, Field
from typing import Optional, List


class Project(BaseModel):
    project_name: str = Field(..., description="Name of the project")
    github_link: Optional[HttpUrl] = Field(None, description="GitHub URL of the project")
    live_link: Optional[HttpUrl] = Field(None, description="Live demo or deployment link")
    time_period: Optional[str] = Field(None, description="Time duration or dates of the project")
    features: Optional[List[str]] = Field(default_factory=list, description="Key features of the project")
    tech_stack: Optional[List[str]] = Field(default_factory=list, description="Technologies used in the project")


class SkillsSection(BaseModel):
    programming_languages: Optional[List[str]] = Field(default_factory=list, description="Languages like Python, C++")
    frameworks: Optional[List[str]] = Field(default_factory=list, description="Frameworks like Django, React")
    libraries_tools: Optional[List[str]] = Field(default_factory=list, description="Libraries and tools like NumPy, Git")
    databases: Optional[List[str]] = Field(default_factory=list, description="Databases like MongoDB, MySQL")
    soft_skills: Optional[List[str]] = Field(default_factory=list, description="Soft skills like leadership, communication")


class ExtractedResumeData(BaseModel):
    name: Optional[str] = Field(None, description="Candidate's full name")
    email: Optional[EmailStr] = Field(None, description="Email address")
    mob_no: Optional[str] = Field(None, description="Mobile phone number")

    linkedin: Optional[HttpUrl] = Field(None, description="LinkedIn profile URL")
    github: Optional[HttpUrl] = Field(None, description="GitHub profile URL")
    leetcode: Optional[HttpUrl] = Field(None, description="LeetCode profile URL")

    skills: Optional[SkillsSection] = Field(default_factory=SkillsSection, description="Structured skill information")
    projects: Optional[List[Project]] = Field(default_factory=list, description="List of personal or academic projects")

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(model=os.getenv("GROQ_MODEL_NAME"))
llm_structured = llm.with_structured_output(ExtractedResumeData)

In [11]:
resume_data = llm_structured.invoke(resume_text_data)

In [13]:
print(resume_data)

name='Pandharinath Maske' email='pandharinath1437@gmail.com' mob_no='+91 8010922929' linkedin=HttpUrl('https://www.linkedin.com/in/pandharinath-maske-538381289') github=HttpUrl('https://github.com/Pandharimaske') leetcode=HttpUrl('https://leetcode.com/u/PandhariMaske1437/') skills=SkillsSection(programming_languages=['Python', 'C', 'C++'], frameworks=['TensorFlow', 'PyTorch', 'FastAPI', 'LangChain', 'LangGraph', 'Hugging Face'], libraries_tools=['Scikit-learn', 'Pandas', 'NumPy', 'Matplotlib', 'MLflow', 'Docker', 'Git'], databases=['MySQL', 'MongoDB', 'Supabase'], soft_skills=['Problem-Solving', 'Research', 'Adaptability', 'Communication']) projects=[Project(project_name='AI-Powered Coffee Shop Chatbot', github_link=HttpUrl('https://github.com/Pandharimaske/Coffee_Shop_ChatBot'), live_link=HttpUrl('https://coffee-shop-chatbot-p7tr.onrender.com/'), time_period='June 2025 – Present', features=['Modular multi-agent chatbot', 'Custom tools for menu retrieval, order management, and user int

In [16]:
state = ResumeAgentState(**resume_data.model_dump())

In [18]:
for s in state:
    print(s)

('name', 'Pandharinath Maske')
('email', 'pandharinath1437@gmail.com')
('mob_no', '+91 8010922929')
('linkedin', HttpUrl('https://www.linkedin.com/in/pandharinath-maske-538381289'))
('github', HttpUrl('https://github.com/Pandharimaske'))
('leetcode', HttpUrl('https://leetcode.com/u/PandhariMaske1437/'))
('skills', SkillsSection(programming_languages=['Python', 'C', 'C++'], frameworks=['TensorFlow', 'PyTorch', 'FastAPI', 'LangChain', 'LangGraph', 'Hugging Face'], libraries_tools=['Scikit-learn', 'Pandas', 'NumPy', 'Matplotlib', 'MLflow', 'Docker', 'Git'], databases=['MySQL', 'MongoDB', 'Supabase'], soft_skills=['Problem-Solving', 'Research', 'Adaptability', 'Communication']))
('projects', [Project(project_name='AI-Powered Coffee Shop Chatbot', github_link=HttpUrl('https://github.com/Pandharimaske/Coffee_Shop_ChatBot'), live_link=HttpUrl('https://coffee-shop-chatbot-p7tr.onrender.com/'), time_period='June 2025 – Present', features=['Modular multi-agent chatbot', 'Custom tools for menu re